# Test Transform Module

In [1]:
# To DO :
# - combine all tables
# - add hydro table stuff
# - work on aggregation stuff and manual overrides for this

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Standard libraries
import logging
import sys
import os
import pathlib
import random

import pandas as pd
import numpy as np
import sqlalchemy as sa

# Local libraries
import pudl
from pudl.analysis.fill_ferc1_fuel_gaps import *
from pudl.analysis.flag_ferc1_totals import *
from pudl.analysis.clean_combine_ferc1 import *

# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

ferc_engine = sa.create_engine(pudl_settings['ferc1_db'])

In [5]:
# View random utility groups! (thanks, Trenton)
def view_random_utility(df):
    df = df.reset_index(drop=True)
    util_groups = df.groupby(['utility_id_ferc1'])
    random_utility = random.choice(list(util_groups.groups.keys()))
    return df.iloc[util_groups.groups[random_utility]]

In [6]:
# # Establish table connections
steam = pudl_out.plants_steam_ferc1().assign(primary_fuel=np.nan)
# small_plants = pd.read_sql("f1_gnrt_plant", ferc_engine) # RAW
hydro = pudl_out.plants_hydro_ferc1()
pumped_storage = pudl_out.plants_pumped_storage_ferc1()

fbp = pudl_out.fbp_ferc1()

# Get raw version of small plants :) 
small_plants_raw = pd.read_sql("f1_gnrt_plant", ferc_engine)
small_plants = pudl_out.plants_small_ferc1()#.dropna(subset=['plant_name_ferc1'])
# Here we create a fake raw dfs dictionary with just the small plants df to run it through
# Zane's existing transform feature.
fake_dict = {'plants_small_ferc1': small_plants_raw}
new_dict = {}
small_plants_dict = pudl.transform.ferc1.plants_small(fake_dict, new_dict)
small_plants = small_plants_dict['plants_small_ferc1']

# drop rows with no plant name because we can't use that
small_plants = small_plants.dropna(subset=['plant_name_original'])

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
# Read in MUL
import pickle

with open('/Users/aesharpe/Desktop/ferc1_to_eia_full.pkl', 'rb') as handle:
    mul_ferc = pickle.load(handle)

In [10]:
#mul_ferc.columns.tolist()

In [13]:
mul_ferc.record_id_ferc1.duplicated().value_counts()

False    27592
True      6868
Name: record_id_ferc1, dtype: int64

In [16]:
mul_ferc.plant_part.unique()

array(['plant', 'plant_prime_mover', 'plant_unit', 'plant_gen',
       'plant_prime_fuel', 'plant_technology', 'plant_ferc_acct', nan],
      dtype=object)

In [12]:
mul_ferc[mul_ferc.record_id_ferc1.duplicated(keep=False)].sort_values('record_id_ferc1').head(2).T.head(40)
mul_ferc.head(2).T.head(40)

,0,1
record_id_ferc1,f1_steam_2009_12_100_0_1,f1_steam_2009_12_100_0_4
record_id_eia,6641_2009_plant_owned_12685,2051_2009_plant_total_12685
record_count,1.0,1.0
unit_id_pudl,<NA>,<NA>
generator_id,NaN,NaN
energy_source_code_1,SUB,NG
ownership_dupe,False,False
appro_part_label,plant,plant
prime_mover_code,ST,ST
fraction_owned,0.25,1.0


In [28]:
ferc1_tables = {'steam': steam, 'small': small_plants, 'hydro': hydro, 'pumped': pumped_storage}

In [60]:
steam_ferc = SteamTable('steam', ferc1_tables['steam'].copy(), steam_value_cols, pudl_out)
small_ferc = SmallTable('small', ferc1_tables['small'].copy(), small_value_cols)
hydro_ferc = HydroTable('hydro', ferc1_tables['hydro'].copy(), hydro_value_cols)
ps_ferc = HydroTable('pumped-storage', ferc1_tables['pumped'].copy(), pumped_value_cols)
table_list = [steam_ferc, small_ferc, hydro_ferc, ps_ferc]

In [61]:
for table in table_list:
    table.transform()


*** TRANSFORMING STEAM TABLE ***
Cleaning steam table
Flagging totals rows for steam table
 - flagging specific totals
 - adding manual totals
 - backfilling totals by capacity
Labeling fuel types for steam table
 - loading EIA table
 - adding fuel types
  * filling fuels with obvious names
    27072 / 29102 rows left unfilled
  * filling in primary fuel by mmbtu
    3872 / 29102 rows left unfilled
  * filling in eia plants with one reported fuel
    2566 / 29105 rows left unfilled
  * filling in primary fuel by cost
    2290 / 29105 rows left unfilled
  * filling in raw ferc1 fuels
    2132 / 29105 rows left unfilled
  * filling in ferc plants with one fuel
    1318 / 29105 rows left unfilled
  * filling in pudl plants with one fuel
    1189 / 29105 rows left unfilled
  * filling in manually mapped fuels
    1100 / 29105 rows left unfilled
  * front and backfilling values with the same ferc1 id
    842 / 29105 rows left unfilled
  * flipping single fuel outliers for plant_id_ferc1
  

100%|██████████| 1880/1880 [00:38<00:00, 48.76it/s]

 - validating clump findings
Assigning headers to groups


 - likely headers that have not been mapped: ['hydraulic' 'other:' 'other production:' 'lewiston canal facilities:'
 'other' 'hydraulic (1):' 'hydraulic:'
 'other general ops. supervision & engineering' 'other-leased:'
 'renewables' 'renewables:']
 - creating header groups
 - assigning headers to groups
Labeling all obvious headers
 - labeling all headers with a tech name in their name
 - labeling all records with a ferc license hydro

header matches manual plant type: 4902
total manual plant types: 6534
total headers mapped: 14892
total headers with manual: 15837
total rows: 17356


*** TRANSFORMING HYDRO TABLE ***
Cleaning hydro table
Flagging totals rows for hydro table
 - using basic total flag
Labeling fuel types for hydro table

*** TRANSFORMING PUMPED-STORAGE TABLE ***
Cleaning pumped-storage table
Flagging totals rows for pumped-storage table
 - using basic total flag
Labeling fuel types for pumped-storage table


In [46]:
steam_ferc.dropped_cols

""


In [56]:
z = pd.DataFrame()
z = z.append(test)
z

,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw,primary_fuel
0,1994,1,7,AEP Generating Company,531,1108,rockport unit 1,NaN,NaN,0.819843,...,NaN,396788.0,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_1,NaN,NaN
1,1994,1,7,AEP Generating Company,531,1109,rockport unit 2,NaN,NaN,0.781755,...,NaN,230300.0,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_2,NaN,NaN
2,1994,1,7,AEP Generating Company,531,2211,rockport,NaN,NaN,0.800799,...,NaN,627088.0,NaN,1300.0,NaN,NaN,steam,f1_steam_1994_12_1_0_3,NaN,NaN
3,1994,1,7,AEP Generating Company,531,1142,rockport total plant,NaN,462.0,0.781224,...,NaN,1254169.0,NaN,2600.0,NaN,NaN,steam,f1_steam_1994_12_1_0_4,NaN,NaN
4,1996,1,7,AEP Generating Company,531,1142,rockport total plant,NaN,325.0,NaN,...,NaN,1111411.0,NaN,2600.0,NaN,NaN,steam,f1_steam_1996_12_1_0_4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29265,2019,531,582,basin electric power cooperative (pudl determi...,6078,2070,pioneer generation s,NaN,3.0,0.239124,...,NaN,1026326.0,NaN,135.0,NaN,3541.0,combustion_turbine,f1_steam_2019_12_531_1_4,242.0,NaN
29266,2019,531,582,basin electric power cooperative (pudl determi...,2527,2071,spirit mound st,NaN,1.0,0.005209,...,NaN,225689.0,NaN,NaN,NaN,54.0,unknown,f1_steam_2019_12_531_1_5,120.0,NaN
29267,2019,531,582,basin electric power cooperative (pudl determi...,5149,2072,culbertson generatio,426475.0,3.0,0.139795,...,NaN,111163.0,NaN,95.0,NaN,2715.0,combustion_turbine,f1_steam_2019_12_531_2_1,95.0,NaN
29268,2019,531,582,basin electric power cooperative (pudl determi...,15392,2073,wisdom unit 2 genera,NaN,5.0,0.007125,...,NaN,22997.0,NaN,83.0,NaN,352.0,combustion_turbine,f1_steam_2019_12_531_2_2,127.0,NaN


In [51]:
test = steam_ferc.df.copy()
#test[test['record_id'].str.contains('f1_steam_1994_12_1_0_3')]

In [23]:
test

,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw,primary_fuel,total_type
177,1994,2,18,ALABAMA POWER COMPANY,11536,8,ala.elec. coop.,NaN,NaN,0.465541,...,45931.0,NaN,NaN,NaN,NaN,unknown,f1_steam_1994_12_2_1_5,NaN,NaN,None
19798,1997,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,65059.0,NaN,NaN,NaN,NaN,unknown,f1_steam_1997_12_2_2_3,NaN,NaN,None
19799,1998,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,72191.0,NaN,NaN,NaN,NaN,unknown,f1_steam_1998_12_2_2_3,NaN,NaN,None
19800,1999,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,86698.0,NaN,NaN,NaN,NaN,unknown,f1_steam_1999_12_2_2_3,NaN,NaN,None
19801,2000,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,123522.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2000_12_2_2_3,NaN,NaN,None
19802,2001,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,141513.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2001_12_2_2_3,NaN,NaN,None
19803,2002,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,143419.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2002_12_2_2_3,NaN,NaN,None
19804,2003,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,154115.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2003_12_2_2_3,NaN,NaN,None
19805,2004,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,144806.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2004_12_2_2_3,NaN,NaN,None
19806,2005,2,18,ALABAMA POWER COMPANY,8468,1380,al elec coop's port.,NaN,NaN,inf,...,134083.0,NaN,NaN,NaN,NaN,unknown,f1_steam_2005_12_2_2_3,NaN,NaN,None


In [21]:
file_path = '/Users/aesharpe/Desktop/manual_total_types.xlsx'
col_name = 'total_type_manual'

#add_manual_values(test, col_name, file_path)
add_man_totals(test)

adding manual totals


AssertionError: the following record_ids are not in the steam table: ['f1_steam_1994_12_1_0_3', 'f1_steam_1995_12_1_0_3', 'f1_steam_1996_12_1_0_3', 'f1_steam_1997_12_1_0_3', 'f1_steam_1998_12_1_0_3', 'f1_steam_1999_12_1_0_3', 'f1_steam_2000_12_1_0_3', 'f1_steam_2001_12_1_0_3', 'f1_steam_2002_12_1_0_3', 'f1_steam_2003_12_1_0_3', 'f1_steam_2004_12_1_0_3', 'f1_steam_2005_12_1_0_3', 'f1_steam_2006_12_1_0_3', 'f1_steam_2007_12_1_0_3', 'f1_steam_2008_12_1_0_3', 'f1_steam_2009_12_1_0_3', 'f1_steam_2010_12_1_0_3', 'f1_steam_2011_12_1_0_3', 'f1_steam_2012_12_1_0_3', 'f1_steam_2013_12_1_0_3', 'f1_steam_2014_12_1_0_3', 'f1_steam_2015_12_1_0_3', 'f1_steam_2016_12_1_0_3', 'f1_steam_2017_12_1_0_3', 'f1_steam_2018_12_1_0_3', 'f1_steam_2019_12_1_0_3', 'f1_steam_1994_12_73_1_3', 'f1_steam_1995_12_73_1_3', 'f1_steam_1996_12_73_1_3', 'f1_steam_1997_12_73_0_3', 'f1_steam_1998_12_73_0_3', 'f1_steam_1999_12_73_0_3', 'f1_steam_2000_12_73_0_3', 'f1_steam_2001_12_73_0_3', 'f1_steam_2002_12_73_0_3', 'f1_steam_2

In [24]:
problem_list = (
    [x for x in list(full_years['record_id']) if x not in list(test['record_id'])])
assert len(problem_list) == 0, \
    f"the following record_ids are not in the steam table: {problem_list}"

In [25]:
# Check that there are no duplicate record_ids
record_dups = list(full_years[full_years['record_id'].duplicated()]['record_id'])
assert len(record_dups) == 0, \
    f"the following record_ids are duplicated: {record_dups}"

In [32]:
steam_ferc.flag_totals()

Flagging totals rows for steam table
flagging specific totals
adding manual totals
backfilling totals by capacity


In [ ]:
# TOTALS and AGG Stuff

In [48]:
steam_ferc.df.total_type.unique()

array([None, 'plant total', 'utility owned extra', 'utility owned total',
       'utility owned steam extra',
       'utility owned combustion turbine extra',
       'utility owned nuclear extra'], dtype=object)

In [38]:
aa = pd.read_excel('/Users/aesharpe/Desktop/manual_total_types.xlsx')

In [39]:
bb = aa.assign(report_year=lambda x: ([list(range(start, end)) for start, end in x[['start_year', 'end_year']].values]))

In [41]:
cc = bb.explode('report_year')

In [45]:
dd = cc.assign(record_id=lambda x: ('f1_steam_' + x.report_year.astype('str') + x.id_suffix))[['record_id', 'total_type_manual']].copy()

In [47]:
dd.total_type_manual.unique()

array(['utility owned plant total', 'plant total', 'unit total',
       'utility owned plant total steam', 'utility owned total',
       'utility owned subtotal', 'utility owned total nuclear',
       'utility owned plant extra'], dtype=object)

In [50]:
problem_list = (
    [x for x in list(dd['record_id']) if x not in list(steam_ferc.df['record_id'])])
assert len(problem_list) == 0, \
    f"the following record_ids are not in the steam table: {problem_list}"

In [51]:
test = steam_ferc.df.copy()

In [52]:
with_new_values = pd.merge(test, dd, on='record_id', how='left')

In [53]:
with_new_values

,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw,primary_fuel,total_type,total_type_manual
0,1994,1,7,AEP Generating Company,531,1108,rockport unit 1,NaN,NaN,0.819843,...,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_1,NaN,NaN,None,NaN
1,1994,1,7,AEP Generating Company,531,1109,rockport unit 2,NaN,NaN,0.781755,...,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_2,NaN,NaN,None,NaN
2,1994,1,7,AEP Generating Company,531,2211,rockport,NaN,NaN,0.800799,...,NaN,1300.0,NaN,NaN,steam,f1_steam_1994_12_1_0_3,NaN,NaN,None,utility owned plant total
3,1994,1,7,AEP Generating Company,531,1142,rockport total plant,NaN,462.0,0.781224,...,NaN,2600.0,NaN,NaN,steam,f1_steam_1994_12_1_0_4,NaN,NaN,plant total,NaN
4,1996,1,7,AEP Generating Company,531,1142,rockport total plant,NaN,325.0,NaN,...,NaN,2600.0,NaN,NaN,steam,f1_steam_1996_12_1_0_4,NaN,NaN,plant total,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29265,2019,531,582,basin electric power cooperative (pudl determi...,6078,2070,pioneer generation s,NaN,3.0,0.239124,...,NaN,135.0,NaN,3541.0,combustion_turbine,f1_steam_2019_12_531_1_4,242.0,NaN,None,NaN
29266,2019,531,582,basin electric power cooperative (pudl determi...,2527,2071,spirit mound st,NaN,1.0,0.005209,...,NaN,NaN,NaN,54.0,unknown,f1_steam_2019_12_531_1_5,120.0,NaN,None,NaN
29267,2019,531,582,basin electric power cooperative (pudl determi...,5149,2072,culbertson generatio,426475.0,3.0,0.139795,...,NaN,95.0,NaN,2715.0,combustion_turbine,f1_steam_2019_12_531_2_1,95.0,NaN,None,NaN
29268,2019,531,582,basin electric power cooperative (pudl determi...,15392,2073,wisdom unit 2 genera,NaN,5.0,0.007125,...,NaN,83.0,NaN,352.0,combustion_turbine,f1_steam_2019_12_531_2_2,127.0,NaN,None,NaN


In [16]:
eia = pudl_out.gens_eia860().copy()
eia['report_year'] = eia['report_date'].dt.year

In [17]:
eia['tech_diff'] = (
    eia.groupby(['plant_id_eia', 'generator_id',])['technology_description']
    .transform(lambda x: len(x.dropna().unique()) > 1)
)

In [46]:
# View plants that change technology description
import random

regs = ['report_date','plant_id_eia','plant_id_pudl','plant_name_eia', 'generator_id', 'technology_description', 'operating_date',
        'current_planned_operating_date', 'energy_source_code_1', 'fuel_type_code_pudl', 'energy_source_code_2', 'multiple_fuels', 'planned_modifications',
        'operating_switch', 'operational_status', 'planned_modifications', 'planned_repower_date']

aa = eia[eia['tech_diff']==True].copy()
bb = aa.drop_duplicates(subset=['plant_id_eia', 'generator_id']).copy().reset_index()

idx = bb.iloc[random.randint(0,len(bb)-1)]['index']
plt_id = eia.loc[eia.index==idx]['plant_id_eia'].item()
gen_id = eia.loc[eia.index==idx]['generator_id'].item()
eia[(eia['plant_id_eia']==plt_id) & (eia['generator_id']==gen_id)][regs]

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,generator_id,technology_description,operating_date,current_planned_operating_date,energy_source_code_1,fuel_type_code_pudl,energy_source_code_2,multiple_fuels,planned_modifications,operating_switch,operational_status,planned_modifications,planned_repower_date
354707,2004-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,None,None,None,existing,None,None
335129,2005-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,None,None,None,existing,None,None
315225,2006-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,None,None,existing,None,None
294796,2007-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,True,None,existing,True,None
273487,2008-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,False,None,existing,False,None
251659,2009-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,False,None,existing,False,None
230969,2010-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,False,None,existing,False,None
209570,2011-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,False,None,existing,False,None
187764,2012-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,OBG,gas,NG,True,False,None,existing,False,None
165181,2013-01-01,56134,4820,Stockton Regional Water Control Facility,101,None,2000-11-01,None,NG,gas,OBG,None,None,None,existing,None,None


In [67]:
plt_id.item()

55245

In [63]:
[x for x in aa.index.tolist() if x in [135390]]
aa.loc[aa.index==idx]

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,report_year,tech_diff
135390,2014-01-01,59701,10753,Subase Microgrid Project,4180,1317,Connecticut Mun Elec Engy Coop,SMP2,False,ISNE,...,NaN,None,None,None,None,2.0,NaN,06340,2014,True


In [45]:
bb[bb['index']==183776]
#aa.iloc[183776]
aa

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,report_year,tech_diff
373853,2004-01-01,3,32,Barry,195,18,Alabama Power Co,1,False,SOCO,...,0.0,None,None,None,None,138.0,NaN,36512,2004,True
373852,2004-01-01,3,32,Barry,195,18,Alabama Power Co,2,False,SOCO,...,0.0,None,None,None,None,139.0,NaN,36512,2004,True
373840,2004-01-01,7,207,Gadsden,195,18,Alabama Power Co,1,True,SOCO,...,0.0,None,None,None,None,64.0,NaN,35903,2004,True
373839,2004-01-01,7,207,Gadsden,195,18,Alabama Power Co,2,True,SOCO,...,0.0,None,None,None,None,66.0,NaN,35903,2004,True
373832,2004-01-01,10,250,Greene County,195,18,Alabama Power Co,1,False,SOCO,...,0.0,None,None,None,None,254.0,NaN,36732,2004,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,2019-01-01,61364,11404,Hu Honua Bioenergy Facility,61001,<NA>,"Hu Honua Bioenergy, LLC",HHB,False,HECO,...,NaN,None,None,None,None,32.0,NaN,96783,2019,True
3118,2019-01-01,61416,11456,Mount Sinai Hospital,61053,5793,Mount Sinai Hospital,EB10,False,NYIS,...,NaN,None,None,False,None,1.5,NaN,10029,2019,True
3117,2019-01-01,61416,11456,Mount Sinai Hospital,61053,5793,Mount Sinai Hospital,EB11,False,NYIS,...,NaN,None,None,False,None,1.5,NaN,10029,2019,True
2529,2019-01-01,61846,12827,Tulare BioMAT Fuel Cell,61467,6492,"Central CA Fuel Cell 2, LLC",MM27,None,CISO,...,NaN,None,None,False,None,2.8,NaN,93274,2019,True


In [33]:
aa

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,report_year,tech_diff
373853,2004-01-01,3,32,Barry,195,18,Alabama Power Co,1,False,SOCO,...,0.0,None,None,None,None,138.0,NaN,36512,2004,True
373852,2004-01-01,3,32,Barry,195,18,Alabama Power Co,2,False,SOCO,...,0.0,None,None,None,None,139.0,NaN,36512,2004,True
373840,2004-01-01,7,207,Gadsden,195,18,Alabama Power Co,1,True,SOCO,...,0.0,None,None,None,None,64.0,NaN,35903,2004,True
373839,2004-01-01,7,207,Gadsden,195,18,Alabama Power Co,2,True,SOCO,...,0.0,None,None,None,None,66.0,NaN,35903,2004,True
373832,2004-01-01,10,250,Greene County,195,18,Alabama Power Co,1,False,SOCO,...,0.0,None,None,None,None,254.0,NaN,36732,2004,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,2019-01-01,61364,11404,Hu Honua Bioenergy Facility,61001,<NA>,"Hu Honua Bioenergy, LLC",HHB,False,HECO,...,NaN,None,None,None,None,32.0,NaN,96783,2019,True
3118,2019-01-01,61416,11456,Mount Sinai Hospital,61053,5793,Mount Sinai Hospital,EB10,False,NYIS,...,NaN,None,None,False,None,1.5,NaN,10029,2019,True
3117,2019-01-01,61416,11456,Mount Sinai Hospital,61053,5793,Mount Sinai Hospital,EB11,False,NYIS,...,NaN,None,None,False,None,1.5,NaN,10029,2019,True
2529,2019-01-01,61846,12827,Tulare BioMAT Fuel Cell,61467,6492,"Central CA Fuel Cell 2, LLC",MM27,None,CISO,...,NaN,None,None,False,None,2.8,NaN,93274,2019,True


In [25]:
gen_id

'GEN3'

In [40]:
regs = ['report_date','plant_id_eia','plant_id_pudl','plant_name_eia', 'generator_id', 'technology_description', 'operating_date']
eia[(eia['plant_id_eia']==3) & (eia['report_date'].dt.year==2004)][regs]
eia[(eia['plant_id_eia']==3) & (eia['generator_id']=='1')][regs]

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,generator_id,technology_description,operating_date
373853,2004-01-01,3,32,Barry,1,None,1954-02-01
354361,2005-01-01,3,32,Barry,1,None,1954-02-01
334577,2006-01-01,3,32,Barry,1,None,1954-02-01
314336,2007-01-01,3,32,Barry,1,None,1954-02-01
293076,2008-01-01,3,32,Barry,1,None,1954-02-01
271362,2009-01-01,3,32,Barry,1,None,1954-02-01
249119,2010-01-01,3,32,Barry,1,None,1954-02-01
227763,2011-01-01,3,32,Barry,1,None,1954-02-01
205895,2012-01-01,3,32,Barry,1,None,1954-02-01
183235,2013-01-01,3,32,Barry,1,None,1954-02-01


In [122]:
interest_cols = ['net_generation_mwh', 'capacity_mw', 'opex_fuel', 'opex_fuel_per_mwh', 'capacity_factor', 
'installation_year']

In [123]:
for col in interest_cols:
    show_tots(col)

NET_GENERATION_MWH 

isna = True:
 True     264
False    221
Name: net_generation_mwh, dtype: int64
total sum: 2044124730.0
full sum: 51372828343.0
pct of total: 4.0

is zero = True:
 False    485
Name: net_generation_mwh, dtype: int64
total sum: 0.0
full sum: 51372828343.0
pct of total: 0.0

CAPACITY_MW 

isna = True:
 False    485
Name: capacity_mw, dtype: int64
total sum: 369990.0
full sum: 51372828343.0
pct of total: 0.0

is zero = True:
 True     262
False    223
Name: capacity_mw, dtype: int64
total sum: 0.0
full sum: 51372828343.0
pct of total: 0.0

OPEX_FUEL 

isna = True:
 False    306
True     179
Name: opex_fuel, dtype: int64
total sum: 30875491085.0
full sum: 51372828343.0
pct of total: 60.1

is zero = True:
 False    485
Name: opex_fuel, dtype: int64
total sum: 0.0
full sum: 51372828343.0
pct of total: 0.0

OPEX_FUEL_PER_MWH 

isna = True:
 True     297
False    188
Name: opex_fuel_per_mwh, dtype: int64
total sum: 184153.0
full sum: 51372828343.0
pct of total: 0.0

is zero

In [42]:
steam.columns.tolist()
steam.construction_type.unique()

array(['conventional', 'unknown', 'outdoor', 'semioutdoor'], dtype=object)

In [24]:
eia.columns.tolist()

['report_date',
 'plant_id_eia',
 'plant_id_pudl',
 'plant_name_eia',
 'utility_id_eia',
 'utility_id_pudl',
 'utility_name_eia',
 'generator_id',
 'associated_combined_heat_power',
 'balancing_authority_code_eia',
 'balancing_authority_name_eia',
 'bypass_heat_recovery',
 'capacity_mw',
 'carbon_capture',
 'city',
 'cofire_fuels',
 'county',
 'current_planned_operating_date',
 'data_source',
 'deliver_power_transgrid',
 'distributed_generation',
 'duct_burners',
 'energy_source_1_transport_1',
 'energy_source_1_transport_2',
 'energy_source_1_transport_3',
 'energy_source_2_transport_1',
 'energy_source_2_transport_2',
 'energy_source_2_transport_3',
 'energy_source_code_1',
 'energy_source_code_2',
 'energy_source_code_3',
 'energy_source_code_4',
 'energy_source_code_5',
 'energy_source_code_6',
 'ferc_cogen_status',
 'ferc_exempt_wholesale_generator',
 'ferc_small_power_producer',
 'fluidized_bed_tech',
 'fuel_type_code_pudl',
 'fuel_type_count',
 'grid_voltage_2_kv',
 'grid_voltag